# Floyd-Warshal APSP

In graph theory, the Floyd-Warshall (FW) algorithms is an All-Pairs Shortest Path (APSP) algorithm. This means it can find the shortest path between all pairs of nodes.

The main idea behind the Floyd-Warshall algorithm is <font color="orange" size="3"><b>to gradually build up all intermediate routes between nodes i and j</b></font> to find the optimal path.

The time complexity to run Floyd-Warshall is <font color="red" size="3"><b>O(V^3)</b></font> which is ideal <u>only for graphs no larger than a couple of hundred nodes </u>. 

**Floyd-Warshall shines** on:
- Small graphs
- All-Pairs Shortest Path problems
- Detecting negative cycles

In [1]:
# directed weighted graph  with negative cycle as an example
graph = {
    0:  [(1, 1), (2, 1)],
    1:  [(3, 4)],        
    2:  [(1, 1)],
    3:  [(2,-6), (4, 1), (5, 1)],
    4:  [(5, 1), (6, 3)],
    5:  [(6, 1)],
    6:  []
}

In [13]:
def setup(graph):
    
    # trasforming graph into adjacency list form to the adjacency matrix form
    adj_matrix = [[float('inf') for _ in range(len(graph))] for _ in range(len(graph))]
    for edge, ver in graph.items():
        for neighbour in ver:
            adj_matrix[edge][neighbour[0]] = neighbour[1]
    
    # dp: memo table that will contain APSP soln
    dp = [[float('inf') for _ in range(len(adj_matrix))] for n in range(len(graph))]
    
    # next_: matrix used to reconstruct shortest paths
    next_ = [[float('inf') for _ in range(len(adj_matrix))] for n in range(len(graph))]
    for i in range(len(adj_matrix)):
        for j in range(len(adj_matrix)):
            dp[i][j] = adj_matrix[i][j]
            if adj_matrix[i][j] is not float('inf'):
                next_[i][j] = j
                
    return dp, next_, adj_matrix

# # # # # # # # # # # # # # # # # # # # # # # 

def reconstructPathFW(start, end, dp, next_):
    path = []

    if dp[start][end] == float('inf'): 
        print(f'\n>>> Nodes {start} and {end} are disjointed!')
        return path

    at = start
    path.append(at)
    while at != end:
        at = next_[at][end]
        
        if at == -1:
            print(f'\n>>> Negative Cycle detected in the path between nodes {start} and {end}')
            return None
        if at != end:
            path.append(at)
    
    if next_[at][end] == -1: 
        print(f'\n>>> Negative Cycle detected in the path\n between nodes {start} and {end}')
        return None
    
    path.append(end)
    print(f"\nFound path from node {start} to node {end}!\nPath: {' => '.join(map(str,path))}")
    return path
    
# # # # # # # # # # # # # # # # # # # # # # #     
 
def propagateNegativeCycles(adj_matrix, dp, next_):
    # execute FW APSP algorithm a second time but
    # this time if the distance can be improved
    # set the optimal distance to be -inf
    # every edge (i,j) marked with -inf is either
    # part of or reaches into a negative cycle
    for k in range(len(adj_matrix)):
        for i in range(len(adj_matrix)):
            for j in range(len(adj_matrix)):
                
                if dp[i][k] + dp[k][i] < dp[i][j]:
                    dp[i][j] = float('-inf')
                    next_[i][j] = -1
    
    return dp, next_

# # # # # # # # # # # # # # # # # # # # # # #  

# execute FW all pairs shortest path algorithm
def floydWarshall(graph, start, end):
    
    dp, next_, adj_matrix = setup(graph)
    
    for k in range(len(adj_matrix)):
        for i in range(len(adj_matrix)):
            for j in range(len(adj_matrix)):
                if dp[i][k] + dp[k][j] < dp[i][j]:
                    dp[i][j] = dp[i][k] + dp[k][j]
                    next_[i][j] = next_[i][k]
    
    #  detect and propagate negative cycles.
    dp, next_ = propagateNegativeCycles(adj_matrix, dp, next_)
    
    # reconstruct the shortest path
    path = reconstructPathFW(start, end, dp, next_)
    
    # return APSP matrix, the shortest path.
    return dp, path

In [ ]:
dp, path = floydWarshall(graph, 1, 3)


>>> Negative Cycle detected in the path between nodes 1 and 3


In [11]:
dp, path = floydWarshall(graph, 4, 3)


Found path from node 4 to node 6!
Path: 4 => 5 => 6
